In [98]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue
import numpy as np

In [99]:
PROBLEM_SIZE = 15
NUM_SETS = 30
SETS = tuple(np.array([random() < 0.25 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)) #Set are created randomly, 25% true 75% false

State = namedtuple('State', ['taken', 'not_taken'])

In [100]:
def check_super_sets():   #I check if there are sets with all True or all False values
    filtered_sets = tuple(filter(any, SETS)) 
    all_true_sets = tuple(filter(all, SETS)) 
    return filtered_sets, all_true_sets 
 
filtered_sets, all_true_sets = check_super_sets() #filtered sets without sets with all false
if(len(all_true_sets) != 0): 
    print("Problem solvable with only one set") 
NUM_FSETS = len(filtered_sets)  #number of filtered sets without sets with all false
print(NUM_FSETS) 
#print(filtered_sets)

29


In [101]:
def covered(state):
    return reduce(
        np.logical_or,
        [filtered_sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state))

assert goal_check(State(set(range(NUM_FSETS)), set())), "Probelm not solvable"

In [102]:
#The estimates of these heuristics may not be optimal but they provide a reasonable estimate of the number of sets needed to cover the remaining elements.
def h_largest_set(state):
    largest_set_size = max(sum(s) for s in filtered_sets) 
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size) #with this optimistic estimate you hope that the largest_set_size hasn't the same nodes that you have already taken
    return optimistic_estimate


def h2_largest_uncovered_set(state):
    already_covered = covered(state)  #remove the taken nodes before considering the largest_set_size available!
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in filtered_sets)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)  #Calculate an optimistic estimate of the number of sets needed to cover the remaining elements. Division of the number of missing elements by the size of the largest subset 
    return optimistic_estimate


def h3_select_powerful_set(state):   #remove the taken nodes before considering the largest_set_size available and then ordered them descendently!
    already_covered = covered(state)  
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in filtered_sets), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size: #calculate how many of the candidates subsets must be selected (taken) so that the sum of their element-covering capacities is sufficient to cover all the missing elements.
        taken += 1                                #increments the value of taken until the sum of the first taken subsets is sufficient to cover all the missing elements.
    return taken
# With h3 as heuristic function we obtain an improvement of 4 orders of magnitude compared to breadth first


#h4 is halfway between h2 and h3. The main difference is that h3 sorts the subsets in descending order by the number of uncovered elements that each subset covers,
#while h4 sorts the subsets in descending order by the total number of elements covered by each subset(as h2). But at the end use the same strategy of h3 with the variable taken
#This function is slower than h3 and h2, and it can take also 2 minutes for unlucky sets (using PROBLEM_SIZE=20 and NUM_SETS=40).It also shows how write code usign numpy arrays help the performance
def h4_max_elements_covered(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    
    elements_covered_by_set = [sum(s) for s in filtered_sets] # Calculate the number of elements covered by each subset
    sorted_sets = sorted(enumerate(elements_covered_by_set), key=lambda x: x[1], reverse=True) #Sort subsets by number of elements covered (descending)
    
    taken = 0
    total_elements_covered = 0
    
    for i, num_elements_covered in sorted_sets:
        if total_elements_covered + num_elements_covered < missing_size:
            total_elements_covered += num_elements_covered
            taken += 1
        else:
            break
    
    return taken

def f(state):
    return len(state.taken) + h3_select_powerful_set(state)

In [103]:
frontier = PriorityQueue()  
state = State(set(), set(range(NUM_FSETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
    
print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

Solved in 20 steps (3 tiles)


In [104]:
current_state


State(taken={17, 3, 20}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28})